In [1]:
import pandas as pd
import numpy as np
import os 
import re

In [2]:
os.getcwd()

'c:\\Users\\user\\Desktop\\Paper LeL\\Code'

# Dataset Preparation

In [3]:
#reading the dataset from 2nd file
data_white2 = pd.read_csv('..\\Data\\protoroles_eng_ud1.2_11082016.tsv', sep = '\t')
data_white2.shape
data_white2.head()

,Dataset,Is.Pilot,Passes.Filters,Protocol,Split,Annotator.ID,Sentence.ID,Pred.Token,Pred.Lemma,Gram.Func,Arg.Phrase,Arg.Tokens.Begin,Arg.Tokens.End,Property,Response,Applicable,Sent.Grammatical
0,bulkfiltered,False,True,spr2,train,75,en-ud-train.conllu 5620,3,be,nsubj,the main reason,0,2,awareness,1.0,yes,2.0
1,bulkfiltered,False,True,spr2,train,73,en-ud-train.conllu 4472,1,e-mail,dobj,your assistant,2,3,awareness,4.0,yes,5.0
2,bulkfiltered,False,True,spr2,train,94,en-ud-train.conllu 9843,2,find,nsubj,i,1,1,awareness,5.0,yes,5.0
3,bulkfiltered,False,True,spr2,train,56,en-ud-train.conllu 4484,4,exist,nsubj,other &quot; guidelines &quot;,0,3,awareness,1.0,no,5.0
4,bulkfiltered,False,True,spr2,train,75,en-ud-train.conllu 4484,4,exist,nsubj,other &quot; guidelines &quot;,0,3,awareness,1.0,yes,5.0


In [4]:
data_white2 = data_white2[(data_white2["Protocol"] == "spr2") &
                data_white2["Gram.Func"].isin(["nsubj", "dobj"])]

data_col_redux = data_white2[["Split",'Sentence.ID','Pred.Token','Pred.Lemma']]      #,"Arg.Tokens.Begin", "Arg.Tokens.End"]]
data_col_redux

,Split,Sentence.ID,Pred.Token,Pred.Lemma
0,train,en-ud-train.conllu 5620,3,be
1,train,en-ud-train.conllu 4472,1,e-mail
2,train,en-ud-train.conllu 9843,2,find
3,train,en-ud-train.conllu 4484,4,exist
4,train,en-ud-train.conllu 4484,4,exist
...,...,...,...,...
193991,train,en-ud-train.conllu 2767,1,give
193992,train,en-ud-train.conllu 2767,1,give
193993,train,en-ud-train.conllu 2767,1,give
193994,train,en-ud-train.conllu 2767,1,give


In [5]:
#isolate training data
data_train = data_col_redux[data_col_redux["Split"] == "train"]
data_train
                             

,Split,Sentence.ID,Pred.Token,Pred.Lemma
0,train,en-ud-train.conllu 5620,3,be
1,train,en-ud-train.conllu 4472,1,e-mail
2,train,en-ud-train.conllu 9843,2,find
3,train,en-ud-train.conllu 4484,4,exist
4,train,en-ud-train.conllu 4484,4,exist
...,...,...,...,...
193991,train,en-ud-train.conllu 2767,1,give
193992,train,en-ud-train.conllu 2767,1,give
193993,train,en-ud-train.conllu 2767,1,give
193994,train,en-ud-train.conllu 2767,1,give


In [6]:
#convert sentence ids to integers and set them as index
sent_ids = data_train["Sentence.ID"].apply(lambda x: int(x.split()[1])).to_list()
data_train = data_train.drop("Sentence.ID", axis = 1)
data_train.insert(0, "Sentence.ID", sent_ids)
data_train = data_train.set_index("Sentence.ID")
data_train

,Split,Pred.Token,Pred.Lemma
Sentence.ID,,,
5620,train,3,be
4472,train,1,e-mail
9843,train,2,find
4484,train,4,exist
4484,train,4,exist
...,...,...,...
2767,train,1,give
2767,train,1,give
2767,train,1,give


In [7]:
# open a dataframe of ids and sents, previously created
train_sents_df  = pd.read_csv('..\\Data\\ewt-train-sents_2.csv').drop('Unnamed: 0', axis = 1).set_index('Idx')
train_sents_df.head()

,Sent.
Idx,
12,The police commander of Ninevah Province annou...
17,"In Baghdad Kadhim Talal Husain, assistant dean..."
18,"Guerrillas killed an engineer, Asi Ali, from T..."
22,Guerrillas near Hawijah launched an attack tha...
27,The Commission said it had no legal grounds fo...


In [8]:
#merge the sents with the metadata df
merged_train = train_sents_df.merge(data_train, left_index = True, right_index = True)

In [9]:
merged_train

,Sent.,Split,Pred.Token,Pred.Lemma
Idx,,,,
12,The police commander of Ninevah Province annou...,train,6,announce
12,The police commander of Ninevah Province annou...,train,6,announce
12,The police commander of Ninevah Province annou...,train,6,announce
12,The police commander of Ninevah Province annou...,train,6,announce
12,The police commander of Ninevah Province annou...,train,6,announce
...,...,...,...,...
12540,On Monday I called and again it was a big to-d...,train,3,call
12540,On Monday I called and again it was a big to-d...,train,3,call
12540,On Monday I called and again it was a big to-d...,train,3,call


In [10]:
import pickle
with open("C:\\Users\\user\\Desktop\\Mattia\\Tesi\\Tokenized_target_sentences\\tknzd_train.pickle", "rb") as infile:
    tknzd_sents = pickle.load(infile)

print(len(set([i[0] for i in tknzd_sents])))

2226


In [11]:
# convert the list of tuple with (sent_id, tknzd_sent) into a dataframe
tknzd_sents_df = pd.DataFrame(tknzd_sents, columns= ["Idx", "Sent."]).set_index("Idx")
#merge df and tokenized df
# tknxd_merged_df = tknzd_sents_df.merge(data_train, left_index = True, right_index = True)

In [12]:
# aggregating dataframe with full and tokenized sents
complete_df = merged_train.drop_duplicates().merge(tknzd_sents_df, left_index= True, right_index= True)

In [13]:
os.getcwd()

'c:\\Users\\user\\Desktop\\Paper LeL\\Code'

In [14]:
# complete_df.to_csv("./complete_df.tsv", sep = "\t")

In [15]:
# extract the actual words through the provided ids and store them along with their sent.id
words = []
lista_frasi  = complete_df[["Sent._y", "Pred.Token"]].to_records().tolist()
for i in range(len(lista_frasi)):
    target_id = lista_frasi[i][2]
    target_word = lista_frasi[i][1][target_id]
    sent_id = lista_frasi[i][0]
    words.append((sent_id,target_word))

In [16]:
words = sorted(list(set(words)))
print(len(words))

1966


---



In [17]:
from transformers import RobertaTokenizerFast, RobertaModel
bba = "phueb/BabyBERTa-2"
baby_berta_tok = RobertaTokenizerFast.from_pretrained(bba, add_prefix_space = True)

c:\Users\user\anaconda3\envs\paperLeL\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
#word pieces combination based on tokens ids
def combine_subwords(ids, words):
    
    """The function takes in two arguments both obtained with hf TokenizerFast class:\
    ids: a list of successive non-single ids \
    word_pieces: a list of word pieces
    
    return:
    a dictionary mapping the ids to their respective subwords
    a list of the reconstructed words"""
    
    words = list(map(lambda x:x.replace("Ġ", "").strip(), words))
    
    # Ensure both input lists have the same length
    if len(ids) != len(words):
        raise ValueError("Input lists must have the same length")

    # Create a dictionary to store word pieces by id
    id_word_dict = {}

    # Iterate through the lists and populate the dictionary
    for id, word in zip(ids, words):
        if id not in id_word_dict:
            id_word_dict[id] = []
        id_word_dict[id].append(word)

    # TODO adjust the code to take the average vectors

    # Create the list of tuples by joining the word pieces
    result = [(id, ''.join(word_pieces)) for id, word_pieces in id_word_dict.items()]

    return id_word_dict, result[1:]


In [19]:
#extract the untoknized sents from the df
sents = complete_df["Sent._x"].tolist()

#tokenize each sentence with babyberta tokenizer ###make it a function later!!!###
hf_tknzd_sents = []
for s in sents:
  tokenized_sent = baby_berta_tok(s)
  sent_id = tokenized_sent.word_ids()
  word_pieces = tokenized_sent.tokens()
  _, tok2words_mapping = combine_subwords(sent_id, word_pieces)
  fullword_tknzd_sent = [w[1] for w in tok2words_mapping ]

  #append each tokenized word to the list
  hf_tknzd_sents.append(fullword_tknzd_sent)

In [20]:
#chech sentences that don't match in lengths
count = 0
unmatched = []
for i in zip(hf_tknzd_sents, complete_df["Sent._y"].tolist()):
  if len(i[0]) != len(i[1]):
    count+=1
    unmatched.append(i)
print(f" Le lunghezze di {count} frasi non corrispondono tra le due tokenizzazioni")

 Le lunghezze di 429 frasi non corrispondono tra le due tokenizzazioni


In [21]:
#show unmatched tokenization
for i in unmatched:
    print("\n",i[0], "\n", i[1], "\n")


 ['yesterday', 'there', 'were', 'tens', 'of', 'them', 'putting', 'road', 'blocks', 'on', 'our', 'street', 'and', 'setting', 'up', 'mortars', ',', 'they', 'only', 'come', 'out', 'in', 'the', 'open', 'when', 'americans', 'leave', 'the', 'area', ',', 'then', 'they', 'start', 'firing', 'mortars', 'indiscriminately', 'and', 'shooting', 'their', 'ak', '-', '47', "'s", 'in', 'the', 'air', '.'] 
 ['Yesterday', 'there', 'were', 'tens', 'of', 'them', 'putting', 'road', 'blocks', 'on', 'our', 'street', 'and', 'setting', 'up', 'mortars', ',', 'they', 'only', 'come', 'out', 'in', 'the', 'open', 'when', 'Americans', 'leave', 'the', 'area', ',', 'then', 'they', 'start', 'firing', 'mortars', 'indiscriminately', 'and', 'shooting', 'their', 'AK', '-', "47's", 'in', 'the', 'air', '.'] 


 ['there', 'are', 'rumours', 'about', 'preparations', 'by', 'slum', 'dwellers', 'for', 'another', 'looting', 'spree', 'against', 'banks', ',', 'governmental', 'and', 'public', 'property', 'similar', 'to', 'the', 'one', 

In [22]:
#add a column of hf tokenized sents to the dataframe
complete_df["hf_tknzd_sents"] = hf_tknzd_sents

In [23]:
#add the previously extracted words as column to the df
words_df = pd.DataFrame(words, columns= ["Idx", "Token"]).set_index("Idx")
complete_df = complete_df.merge(words_df, right_index= True, left_index= True)
complete_df

,Sent._x,Split,Pred.Token,Pred.Lemma,Sent._y,hf_tknzd_sents,Token
Idx,,,,,,,
12,The police commander of Ninevah Province annou...,train,6,announce,"[The, police, commander, of, Ninevah, Province...","[the, police, commander, of, ninevah, province...",announced
18,"Guerrillas killed an engineer, Asi Ali, from T...",train,1,kill,"[Guerrillas, killed, an, engineer, ,, Asi, Ali...","[guerrillas, killed, an, engineer, ,, asi, ali...",killed
22,Guerrillas near Hawijah launched an attack tha...,train,3,launch,"[Guerrillas, near, Hawijah, launched, an, atta...","[guerrillas, near, hawijah, launched, an, atta...",launched
27,The Commission said it had no legal grounds fo...,train,2,say,"[The, Commission, said, it, had, no, legal, gr...","[the, commission, said, it, had, no, legal, gr...",said
38,Al-Sharq al-Awsat reports that a key eyewitnes...,train,6,report,"[Al, -, Sharq, al, -, Awsat, reports, that, a,...","[al, -, sharq, al, -, awsat, reports, that, a,...",reports
...,...,...,...,...,...,...,...
12512,Other shops around this city have MUCH NICER a...,train,5,have,"[Other, shops, around, this, city, have, MUCH,...","[other, shops, around, this, city, have, much,...",have
12527,"I got the order completed, and then questioned...",train,1,get,"[I, got, the, order, completed, ,, and, then, ...","[i, got, the, order, completed, ,, and, then, ...",got
12528,"He said it was the same tire, and verified thi...",train,1,say,"[He, said, it, was, the, same, tire, ,, and, v...","[he, said, it, was, the, same, tire, ,, and, v...",said


## Tokenizations Alignment

In [24]:
# #extract the relevant columns from the df
sents_and_toks = complete_df[["Sent._y", "Token"]].to_records().tolist()

original_ids = complete_df["Pred.Token"].tolist()

# #check if there are more then one token per sequence of the same word
# def check_matches(lista1, lista2):
#     lens = []
#     indexes = []
#     sents2check = []
#     unmatched_ids = []
#     sent_ids = []
    
#     #unpack the elements for clarity
#     for el in lista1:
#         sent_id = el[0]
#         sent = el[1]
#         tok = el[2]
        
#         #check if there is more than one target token in the corresponding sent
#         l = len([t for t in sent if t== tok])
#         if l>1:
#             lens.append(l)
            
#             # store the sentences with more than one correspondence for the target token
#             sents2check.append((sent_id, sent))
        
#         #store the actual index of the target token in the sent
#         indexes.append(sent.index(tok))
        
#         #re-store sent ids
#         sent_ids.append(sent_id)
    
#     # check correspondences between original ids and actual ids
#     for i in zip(indexes, lista2, sent_ids):
#         if i[0] != i[1]:
#             unmatched_ids.append((i))
    
#     return lens, sents2check, unmatched_ids, indexes
    

In [25]:
def check_matches(lista1, lista2):
    lens = []
    indexes = []
    sents2check = []
    unmatched_ids = []
    sent_ids = []
    dummy_number = 0000
    #unpack the elements for clarity
    for el in lista1:
        sent_id = el[0]
        sent = [w.lower() for w in el[1]]
        tok = el[2].lower()
        
        #check if there is more than one target token in the corresponding sent
        l = len([t for t in sent if t== tok])
        if l>1:
            lens.append(l)
            
            # store the sentences with more than one correspondence for the target token
            sents2check.append((sent_id, sent))
        
        #store the actual index of the target token in the sent
        try: 
            target_id = sent.index(tok) 
            indexes.append(target_id)
        #catch the errors for the "ill-tokenized" words
        except ValueError:
            print(f"{sent_id}\t{tok}")
            indexes.append(dummy_number)
            pass
        
        #re-store sent ids
        sent_ids.append(sent_id)
    
    # check correspondences between original ids and actual ids
    for i in zip(indexes, lista2, sent_ids):
        if i[0] != i[1]:
            unmatched_ids.append((i))
    print(f"The number of sentnces with more than 1 word equal to the target is: {len(lens)}")
    return lens, sents2check, unmatched_ids, indexes

White et al. Tokenization check (mostly to catch disallignment due to index() method)

In [26]:
#check the correspondence between original ids and original tokenization
morethan1tok, sents2check, unmatched_ids,ids = check_matches(sents_and_toks, original_ids)
unmatched_ids

The number of sentnces with more than 1 word equal to the target is: 80


[(7, 25, 1550), (3, 7, 4327), (2, 7, 11576)]

Three elements do not corresponds because the index() method catches the first element and not the actual target (see below)

In [27]:
for i in zip(original_ids, ids):
    if i[0] != i[1]:
        print(i)

(25, 7)
(7, 3)
(7, 2)


In [28]:
#show the sents for wich target token appears more than once and is unmatched
complete_df[complete_df.index.isin([1550, 4327, 11576])]

,Sent._x,Split,Pred.Token,Pred.Lemma,Sent._y,hf_tknzd_sents,Token
Idx,,,,,,,
1550,In the very interview in which they admitted 9...,train,25,admit,"[In, the, very, interview, in, which, they, ad...","[in, the, very, interview, in, which, they, ad...",admitted
4327,The Leads who are responsible for Associates a...,train,7,be,"[The, Leads, who, are, responsible, for, Assoc...","[the, leads, who, are, responsible, for, assoc...",are
11576,"In today's instant world, there's no reason fo...",train,7,be,"[In, today, 's, instant, world, ,, there, 's, ...","[in, today, 's, instant, world, ,, there, 's, ...",'s


HuggiFace Tokenization \
(to build a new list of indexes)

In [29]:
#check the correspondence between original ids and hf tokenization
sents_and_toks_hf = complete_df[["hf_tknzd_sents", "Token"]].to_records().tolist()
morethan1tok2, sents2check2, unmatched_ids_hf, indexes_hf = check_matches(sents_and_toks_hf, original_ids)


4472	e-mailed
5732	re-secured
7377	guz
7411	wan
7895	varie$
9477	’s
9898	<3
10069	s
12024	s
The number of sentnces with more than 1 word equal to the target is: 79


In [30]:
complete_df[complete_df.index.isin([4472, 5732, 7377,7411,7895,9477,9898,100609,12024])]

,Sent._x,Split,Pred.Token,Pred.Lemma,Sent._y,hf_tknzd_sents,Token
Idx,,,,,,,
4472,I e-mailed your assistant earlier this morning...,train,1,e-mail,"[I, e-mailed, your, assistant, earlier, this, ...","[i, e, -, mailed, your, assistant, earlier, th...",e-mailed
5732,My father re-secured the petroleum assets of K...,train,2,re-secure,"[My, father, re-secured, the, petroleum, asset...","[my, father, re, -, secured, the, petroleum, a...",re-secured
7377,iguzwhat lol,train,1,guz,"[i, guz, what, lol]","[iguzwhat, lol]",guz
7411,& I wanna know what food has the game pieces.,train,2,wan,"[&, I, wan, na, know, what, food, has, the, ga...","[&, i, wanna, know, what, food, has, the, game...",wan
7895,It varie$ company to company.,train,1,varie$,"[It, varie$, company, to, company, .]","[it, varie, $, company, to, company, .]",varie$
9477,And then there’s the rain.,train,3,be,"[And, then, there, ’s, the, rain, .]","[and, then, there, âĢĻ, s, the, rain, .]",’s
9898,I <3 Max's,train,1,<3,"[I, <3, Max, 's]","[i, <, 3, max, 's]",<3
12024,Thats how they were caught.,train,1,be,"[That, s, how, they, were, caught, .]","[thats, how, they, were, caught, .]",s


In [31]:
#show all the real mismatches
original_ids_with_sid = complete_df[["Pred.Token"]].to_records().tolist()

count = 0
cc = []
for i in range(len(original_ids)):
    if original_ids[i] != indexes_hf[i]:
        count +=1
        # print(original_ids_with_sid[i])
        cc.append(original_ids_with_sid[i][0])
complete_df[complete_df.index.isin(cc)]

,Sent._x,Split,Pred.Token,Pred.Lemma,Sent._y,hf_tknzd_sents,Token
Idx,,,,,,,
118,As a child in the 50's I had a lot of glandula...,train,7,have,"[As, a, child, in, the, 50's, I, had, a, lot, ...","[as, a, child, in, the, 50, 's, i, had, a, lot...",had
122,The Chernobyl Children's Project (http://www.a...,train,8,offer,"[The, Chernobyl, Children, 's, Project, (, htt...","[the, chernobyl, children, 's, project, (, htt...",offers
125,S. and I have an acquaintance who has hosted s...,train,3,have,"[S., and, I, have, an, acquaintance, who, has,...","[s, ., and, i, have, an, acquaintance, who, ha...",have
164,George W. Bush alleged Thursday that John Edwa...,train,3,allege,"[George, W., Bush, alleged, Thursday, that, Jo...","[george, w, ., bush, alleged, thursday, that, ...",alleged
179,"""Wait, wait, is this 50 questions?"" asked Bush.",train,11,ask,"["", Wait, ,, wait, ,, is, this, 50, questions,...","["", wait, ,, wait, ,, is, this, 50, questions,...",asked
...,...,...,...,...,...,...,...
11576,"In today's instant world, there's no reason fo...",train,7,be,"[In, today, 's, instant, world, ,, there, 's, ...","[in, today, 's, instant, world, ,, there, 's, ...",'s
11769,What I like most about Dr. Liau is that she is...,train,7,be,"[What, I, like, most, about, Dr., Liau, is, th...","[what, i, like, most, about, dr, ., liau, is, ...",is
11869,"At 3:15 am on night #2, the fire alarm and str...",train,14,activate,"[At, 3:15, am, on, night, #, 2, ,, the, fire, ...","[at, 3, :, 15, am, on, night, #, 2, ,, the, fi...",activated


In [32]:
print(f"In total there are {count} mismatches between the ids of the two tokenizations.")

In total there are 135 mismatches between the ids of the two tokenizations.


In [33]:
# extract tokens with the new ids to confront with original
extracted_toks = []
for i in range(len(indexes_hf)) :
    id = indexes_hf[i]
    sent = sents_and_toks_hf[i][1]
    target_tok = sent[id]
    extracted_toks.append(target_tok)

In [34]:
#print the unmatched tokens between the two lists
for i in zip(extracted_toks,complete_df["Token"].tolist()):
    #print(i)
    if i[0]!=i[1].lower():
        print(i)

('i', 'e-mailed')
('my', 're-secured')
('iguzwhat', 'guz')
('&', 'wan')
('it', 'varie$')
('and', '’s')
('i', '<3')
('theres', 's')
('thats', 's')


In [35]:
#TODO remove the sentences with ill-tokenized words and change ids for the 3 with
#TODO add a column to the complete dataframe with the new indexes for the hf tokenization

In [36]:
complete_df["hf_tnzd_ids"] = indexes_hf

In [48]:
complete_df[complete_df.index == 7895]

,Sent._x,Split,Pred.Token,Pred.Lemma,Sent._y,hf_tknzd_sents,Token,hf_tnzd_ids
Idx,,,,,,,,
7895,It varie$ company to company.,train,1,varie$,"[It, varie$, company, to, company, .]","[it, varie, $, company, to, company, .]",varie$,0


In [50]:
ids2remove = [4472, 5732, 7377,7411,7895,9477,9898,12024, 1550, 4327, 11576]
complete_df = complete_df.drop(ids2remove)

In [54]:
# path = "..\\Data"

# with open(os.path.join(path, "complete_df.pkl"), "wb") as outfile:
#     pickle.dump(complete_df, outfile)